In [1]:
import gc
import pickle

import numpy as np
import pandas
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.python.keras import backend
from tensorflow.python.keras.layers import Activation, Dense, Dropout
from tensorflow.python.keras.models import load_model, Sequential
from tensorflow.python.keras.utils.np_utils import to_categorical


### Load Dataset

ar_reviews_100k.csv

In [2]:
dataset = pandas.read_csv("ar_reviews_100k.tsv", sep='\t', header=0)

In [3]:
#print(dataset.head())
#print(type(dataset['text']))

In [4]:
labels = dataset['label']
train_data = dataset.sample(frac=.7)
test_data = dataset.drop(labels=train_data.index)
#print(f"Train Data:{len(train_data)}")
#print(f"Test Data:{len(test_data)}")
train_reviews = train_data['text']
test_reviews = test_data['text']

In [5]:
#print(train_reviews)

In [6]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(dataset['text'])
train_reviews1 = train_reviews[:875]
train_reviews2 = train_reviews[875:1750]
train_reviews3 = train_reviews[1750:2625]
train_reviews4 = train_reviews[2625:3500]
train_reviews5 = train_reviews[3500:4375]
train_reviews6 = train_reviews[4375:5250]
train_reviews7 = train_reviews[5250:6125]
train_reviews8 = train_reviews[6125:6999]
del train_reviews
gc.collect()
tokenized_train1 = tokenizer.texts_to_matrix(train_reviews1, mode='tfidf')
tokenized_train2 = tokenizer.texts_to_matrix(train_reviews2, mode='tfidf')
tokenized_train3 = tokenizer.texts_to_matrix(train_reviews3, mode='tfidf')
tokenized_train4 = tokenizer.texts_to_matrix(train_reviews4, mode='tfidf')
tokenized_train5 = tokenizer.texts_to_matrix(train_reviews5, mode='tfidf')
tokenized_train6 = tokenizer.texts_to_matrix(train_reviews6, mode='tfidf')
tokenized_train7 = tokenizer.texts_to_matrix(train_reviews7, mode='tfidf')
tokenized_train8 = tokenizer.texts_to_matrix(train_reviews8, mode='tfidf')
test_reviews1 = test_reviews[:750]
test_reviews2 = test_reviews[750:1500]
test_reviews3 = test_reviews[1500:2250]
test_reviews4 = test_reviews[2250:3000]
del test_reviews
gc.collect()
tokenized_test1 = tokenizer.texts_to_matrix(test_reviews1, mode='tfidf')
tokenized_test2 = tokenizer.texts_to_matrix(test_reviews2, mode='tfidf')
tokenized_test3 = tokenizer.texts_to_matrix(test_reviews3, mode='tfidf')
tokenized_test4 = tokenizer.texts_to_matrix(test_reviews4, mode='tfidf')


In [8]:
tokenized_train = np.concatenate((tokenized_train1, tokenized_train2, tokenized_train3, tokenized_train4, tokenized_train5, tokenized_train6, tokenized_train7, tokenized_train8), axis=0)
tokenized_test = np.concatenate((tokenized_test1, tokenized_test2, tokenized_test3, tokenized_test4), axis=0)
del tokenized_train1, tokenized_train2, tokenized_train3, tokenized_train4, tokenized_train5, tokenized_train6, tokenized_train7, tokenized_train8
del tokenized_test1, tokenized_test2, tokenized_test3, tokenized_test4
gc.collect()

522

In [9]:
encoder = LabelEncoder()
encoder.fit(labels)
labelst=encoder.fit_transform(labels)
del labels
num_classes = int((len(set(labelst))))
#print((len(set(labelst))))
del labelst
gc.collect()
encoded_train = encoder.fit_transform(train_data['label'])
encoded_test = encoder.fit_transform(test_data['label'])

8lban momkin ashiil num_labels.

we vocab_size we max_words;

bs lw hasiilhom ha7tag a7ot ta7t int(len(tokenizer.word_index) + 1)

In [10]:
encoded_train= to_categorical(encoded_train,num_classes)
encoded_test = to_categorical(encoded_test, num_classes)


num_labels = int(len(encoded_train.shape))
vocab_size = int(len(tokenizer.word_index) + 1)

max_words=vocab_size


In [11]:

def f1_metric(y_true, y_pred):
    true_positives = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = backend.sum(backend.round(backend.clip(y_true, 0, 1)))
    predicted_positives = backend.sum(backend.round(backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + backend.epsilon())
    recall = true_positives / (possible_positives + backend.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+backend.epsilon())
    return f1_val

lw shelt max_word ha7tag a7ot hnaa int(len(tokenizer.word_index) + 1)

bagrab a7ot num_labels bs deh hada mn 3andii

In [12]:
# Build the model
model = Sequential()
#model.add(Dense(1024, input_shape=(max_words,)))
model.add(Dense(1024, input_shape=(max_words,num_labels)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])


In [ ]:
history = model.fit(tokenized_train, encoded_train,
                    batch_size=100,
                    epochs=2,
                    verbose=1,
                    validation_split=0.1)

model.save('my_model.h1')
#del model

In [ ]:


# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

8lban msh m7tgin deh fe 7aga bafakr ashilhaa

hankon m7tgnha fe 7alt enhom m7tagina nkyam el model bs 

In [ ]:
model = load_model('my_model.h1')
Evaluation_valus = model.evaluate(tokenized_test,encoded_test,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)


In [ ]:
for x in dataset["text"][:25]:

    tokens = tokenizer.texts_to_matrix([x], mode='tfidf')

    c=model.predict(np.array(tokens))
    cc=model.predict_classes(tokens)
    xc = encoder.inverse_transform(cc)


    print(c,"= \t",cc,"\t",xc)